In [1]:
%pwd
%cd ../..

/home/rydevera3/data-science/umn_research/Deeplifting


In [2]:
import time
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.optimization import (
    run_deeplifting,
    run_differential_evolution,
    run_dual_annealing,
    run_ipopt,
    run_pygranso,
)

In [3]:
problem_names = [
    'ackley',
    'bukin_n6',
    'cross_in_tray',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
    'cross_leg_table',
    'math'
]

In [4]:
problem_performance_list = []
trials = 10

for problem_name in problem_names:
    print(problem_name)
    # Setup the problem
    problem = PROBLEMS_BY_NAME[problem_name]

    # Get the known minimum
    minimum_value = problem['global_minimum']

    # First run IPOPT
    outputs = run_ipopt(problem, trials=trials)

    # Get the final results for all IPOPT runs
    ipopt_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    ipopt_results['problem_name'] = problem_name
    ipopt_results['hits'] = np.where(
        np.abs(ipopt_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add IPOPT to the problem_performance_list
    problem_performance_list.append(ipopt_results)

    # Next add dual annealing
    outputs = run_dual_annealing(problem, trials=trials)

    # Get the final results for all dual annealing runs
    dual_annleaing_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    dual_annleaing_results['problem_name'] = problem_name
    dual_annleaing_results['hits'] = np.where(
        np.abs(dual_annleaing_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add dual annealing to the problem_performance_list
    problem_performance_list.append(dual_annleaing_results)

    # Next add differential evolution
    outputs = run_differential_evolution(problem, trials=trials)

    # Get the final results for all differential evolution runs
    differential_evolution_results = pd.DataFrame(
        outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    )
    differential_evolution_results['problem_name'] = problem_name
    differential_evolution_results['hits'] = np.where(
        np.abs(differential_evolution_results['f'] - minimum_value) <= 1e-4, 1, 0
    )

    # Add differential evolution to the problem_performance_list
    problem_performance_list.append(differential_evolution_results)

    # Next add pygranso
    # outputs = run_pygranso(problem, trials=trials)

    # # Get the final results for all differential evolution runs
    # pygranso_results = pd.DataFrame(
    #     outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    # )
    # pygranso_results['problem_name'] = problem_name
    # pygranso_results['hits'] = np.where(
    #     np.abs(pygranso_results['f'] - minimum_value) <= 1e-4, 1, 0
    # )

    # # Add differential evolution to the problem_performance_list
    # problem_performance_list.append(pygranso_results)

    # # Finally, add deeplifting
    # outputs = run_deeplifting(problem, trials=trials)

    # # Get the final results for all differential evolution runs
    # deeplifting_results = pd.DataFrame(
    #     outputs['final_results'], columns=['x1', 'x2', 'f', 'algorithm', 'time']
    # )
    # deeplifting_results['problem_name'] = problem_name
    # deeplifting_results['hits'] = np.where(
    #     np.abs(deeplifting_results['f'] - minimum_value) <= 1e-4, 1, 0
    # )

    # # Add differential evolution to the problem_performance_list
    # problem_performance_list.append(deeplifting_results)

ackley

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

bukin_n6
cross_in_tray
drop_wave
eggholder
griewank
holder_table
levy
levy_n13
rastrigin
schaffer_n2
schaffer_n4
schwefel
shubert


In [5]:
results_df = pd.concat(problem_performance_list, ignore_index=True)

In [6]:
results_df

,x1,x2,f,algorithm,time,problem_name,hits
0,-9.684777e-01,-9.684777e-01,3.574452e+00,IPOPT,0.092996,ackley,0
1,-7.669206e-09,9.521665e-01,2.579928e+00,IPOPT,0.012860,ackley,0
2,6.235565e-01,1.346904e+00,5.979170e+00,IPOPT,11.693238,ackley,0
3,-5.285603e-09,-4.220401e-09,1.913099e-08,IPOPT,11.481675,ackley,1
4,-4.635960e-09,-4.136657e-09,1.757363e-08,IPOPT,11.749664,ackley,1
...,...,...,...,...,...,...,...
415,-7.083506e+00,4.858057e+00,-1.867309e+02,Differential Evolution,0.093173,shubert,1
416,4.858057e+00,5.482864e+00,-1.867309e+02,Differential Evolution,0.071135,shubert,1
417,4.858057e+00,-7.083506e+00,-1.867309e+02,Differential Evolution,0.065460,shubert,1
418,-8.003211e-01,4.858057e+00,-1.867309e+02,Differential Evolution,0.084116,shubert,1


In [7]:
performance_df = (
    results_df.groupby(['problem_name', 'algorithm'])
    .agg({'hits': 'mean', 'time': 'sum'})
    .reset_index()
    .rename(columns={'hits': 'Success Rate', 'time': 'Total Run Time'})
)

In [8]:
performance_df

,problem_name,algorithm,Success Rate,Total Run Time
0,ackley,Differential Evolution,1.0,1.778461
1,ackley,Dual Annealing,1.0,2.875483
2,ackley,IPOPT,0.2,35.153245
3,bukin_n6,Differential Evolution,0.0,1.207762
4,bukin_n6,Dual Annealing,0.0,2.633882
5,bukin_n6,IPOPT,0.0,78.000827
6,cross_in_tray,Differential Evolution,1.0,0.263366
7,cross_in_tray,Dual Annealing,1.0,2.622354
8,cross_in_tray,IPOPT,0.9,0.216444
9,drop_wave,Differential Evolution,0.5,0.532001


In [9]:
performance_df = (
    performance_df.pivot_table(
        index='problem_name',
        columns='algorithm',
        values=['Success Rate', 'Total Run Time'],
    )
    .T.reorder_levels(['algorithm', None])
    .sort_index()
    .T
)

In [10]:
performance_df = performance_df.apply(lambda x: np.round(x, 3))

In [ ]:
performance_df.to_csv('/Users/ryandevera/Desktop/deeplifting_results.csv')

In [11]:
performance_df

algorithm     Differential Evolution                Dual Annealing  \
                        Success Rate Total Run Time   Success Rate   
problem_name                                                         
ackley                           1.0          1.778            1.0   
bukin_n6                         0.0          1.208            0.0   
cross_in_tray                    1.0          0.263            1.0   
drop_wave                        0.5          0.532            0.8   
eggholder                        0.2          0.571            0.5   
griewank                         0.7          1.280            0.4   
holder_table                     1.0          0.361            1.0   
levy                             1.0          1.589            1.0   
levy_n13                         1.0          1.593            1.0   
rastrigin                        0.8          1.033            1.0   
schaffer_n2                      1.0          1.193            1.0   
schaffer_n4                      1.0          1.158            1.0   
schwefel                         0.6          0.531            1.0   
shubert                          1.0          0.828            1.0   

algorithm                           IPOPT                 
              Total Run Time Success Rate Total Run Time  
problem_name                                              
ackley                 2.875          0.2         35.153  
bukin_n6               2.634          0.0         78.001  
cross_in_tray          2.622          0.9          0.216  
drop_wave              2.514          0.0          0.169  
eggholder              2.611          0.0          0.165  
griewank               2.566          1.0          0.091  
holder_table           2.562          0.0          0.111  
levy                   2.608          0.9          0.081  
levy_n13               2.607          0.4         11.070  
rastrigin              2.494          0.2          0.147  
schaffer_n2            2.509          1.0          0.143  
schaffer_n4            3.104          0.7          0.246  
schwefel               2.510          0.0          0.097  
shubert                3.407          0.1          1.284